In [1]:
#Tercero Ingredient-Based Reccomender
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import linear_kernel

In [13]:
#Imports Tercero Data
tData = pd.read_csv('df_tercero_all.csv', low_memory = False)

tData.tail(10)

,Unnamed: 0,Name,Location,Mealtime,Date,Filters,Description,Serving_Size,Calories,Fat,Carbohydrates,Protein,Allergens,Ingredients
421,421,Seasonal Student Farm Roasted Vegetable Salad,Harvest Bakery,Dinner,"Saturday, February 05, 2022",['Vegan'],House pasta salad featuring roasted student fa...,1/2 cup,108.59,4.32,14.96,2.59,Gluten/Wheat,FG CSC Dressing Vinaigrette Tomato Basil (CANO...
422,422,Red Skin Potato Salad with Eggs,Harvest Bakery,Dinner,"Saturday, February 05, 2022","['Vegetarian', 'Halal']",NaN,1/2 cup,230.01,13.01,24.00,3.00,Egg/Soybean Oil,"Red potatoes, mayonnaise (soybean oil, water, ..."
423,423,Classic California Pasta Salad,Harvest Bakery,Dinner,"Saturday, February 05, 2022","['Vegan', 'Halal']",NaN,1 serving,140.00,4.00,22.00,4.00,Gluten/Soybean Oil/Wheat,"Tri-color rotini (durum wheat semolina, tomato..."
424,424,CHEF CHOICE SALAD,Harvest Bakery,Dinner,"Saturday, February 05, 2022",[],NaN,1 EA,-1.00,-1.00,-1.00,-1.00,NaN,NaN
425,425,Caesar Salad,Harvest Bakery,Dinner,"Saturday, February 05, 2022",['Halal'],"Traditional Caesar salad with Romaine lettuce,...",1 SV,501.86,41.48,23.93,15.54,Dairy/Gluten/Soybean Oil/Tree Nuts/Wheat/Egg/F...,"Romaine lettuce, Tuscan Caesar dressing (soybe..."
426,426,Muhammara,Harvest Bakery,Dinner,"Saturday, February 05, 2022",['Vegan'],A Middle Eastern spread made with roasted pepp...,2 tbsp,82.85,6.13,7.00,1.06,Gluten/Wheat/Tree Nuts,"Red Bell Pepper, Garlic, Walnuts, Canola/Olive..."
427,427,Black Bean Dip,Harvest Bakery,Dinner,"Saturday, February 05, 2022","['Vegan', 'Nourish']","Flavor-packed bean spread with tomatoes, onion...",2 tbsp,30.47,1.04,4.09,1.44,NaN,"Black Bean, Canola Olive Oil, Salsa (Tomato, J..."
428,428,Tuxedo Truffle Mousse Cake,Harvest Bakery,Dinner,"Saturday, February 05, 2022",['Vegetarian'],"Per manufacturer, may contain peanuts and tree...",1 SV,403.20,26.88,38.98,5.38,Alcohol/Dairy/Egg/Gluten/Soy/Wheat,"Cream, sugar, liquid whole eggs, semi-sweet ch..."
429,429,Oatmeal Cookie,Harvest Bakery,Dinner,"Saturday, February 05, 2022",['Vegetarian'],"Produced in a facility that also uses dairy, e...",1 EA,360.00,14.00,52.00,6.00,Alcohol [vanilla extract]/Egg/Gluten/Soybean O...,"Oats, brown sugar, margarine (palm oil, water,..."
430,430,Vegan Lemon Drop Cookie,Harvest Bakery,Dinner,"Saturday, February 05, 2022",['Vegan'],NaN,1 cookie,157.00,10.00,37.00,2.00,Alcohol [vanilla extract]/Gluten/Wheat,"Unbleached wheat flour, organic evaporated can..."


In [3]:
#This version takes a food, and reccommends similar items based off its contents of ingredients
tData['Ingredients'].head()

0                                                  NaN
1                           Turkey Sausage, Canola Oil
2                           Cage-free eggs, canola oil
3    Potato wedges (potatoes, vegetable oil [soybea...
4                     Cage-free egg whites, canola oil
Name: Ingredients, dtype: object

In [4]:
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
tData['Ingredients'] = tData['Ingredients'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(tData['Ingredients'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(431, 789)

In [5]:
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#Construct a reverse map of indices and food items
indices = pd.Series(tData.index, index=tData['Name']).drop_duplicates()
indices

Name
French Toast                    0
Turkey Sausage Patties          1
Scrambled Eggs                  2
Fries Wedges Spicy              3
Fiesta Scrambler                4
                             ... 
Muhammara                     426
Black Bean Dip                427
Tuxedo Truffle Mousse Cake    428
Oatmeal Cookie                429
Vegan Lemon Drop Cookie       430
Length: 431, dtype: int64

In [6]:
# Function that takes in food item as input and outputs most similar foods in terms of ingredients
def get_recommendations(food, cosine_sim=cosine_sim):
    # Get the index of the food that matches the ingredient
    idx = indices[food]

    # Get the pairwsie similarity scores of all food item with that food item
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the food items based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar food items
    sim_scores = sim_scores[1:11]

    # Get the food item indices
    food_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar food items
    return tData.iloc[food_indices]


In [12]:
get_recommendations("Pork Link Sausage")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()